In [1]:
import pandas as pd
import nltk

In [2]:
df = pd.read_csv("Tweets.csv",encoding="latin-1")

In [5]:
df.shape

(14640, 15)

In [7]:
df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [8]:
df = df.drop(columns=['tweet_id', 'airline_sentiment_confidence', 'negativereason',
                      'negativereason_confidence', 'airline', 'airline_sentiment_gold',
                      'name', 'negativereason_gold', 'retweet_count', 'tweet_coord',
                      'tweet_created', 'tweet_location', 'user_timezone'])


In [10]:
df.head(5)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [13]:
import nltk
import string
import re
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

ps = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = nltk.word_tokenize(text)
    y = []

    for i in text:
        if i not in stopwords.words('english'):
            y.append(ps.stem(i))

    return " ".join(y)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download ('punkt')
ps = PorterStemmer ()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
df['text_cleaned'] = df['text'].apply(clean_text)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Create a TfidfVectorizer with max_features as 3000
tfidf_vectorizer = TfidfVectorizer(max_features=3000)

# Generate TF-IDF vector representation of cleaned text in the column "text_cleaned"
X = tfidf_vectorizer.fit_transform(df['text_cleaned']).toarray()

# Convert the column "airline_sentiment" to an array
Y = df['airline_sentiment'].values


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# Train a multinomial Naïve Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Predict the sentiment of tweets in the test dataset
nb_y_pred = nb_classifier.predict(X_test)

# Find the accuracy of the multinomial Naïve Bayes model
nb_accuracy = accuracy_score(y_test, nb_y_pred)
print("Accuracy of Multinomial Naïve Bayes Classifier:", nb_accuracy)

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)

# Predict the sentiment of tweets in the test dataset using Random Forest
rf_y_pred = rf_classifier.predict(X_test)

# Find the accuracy of the Random Forest model
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print("Accuracy of Random Forest Classifier:", rf_accuracy)


Accuracy of Multinomial Naïve Bayes Classifier: 0.7219945355191257
Accuracy of Random Forest Classifier: 0.7489754098360656


In [19]:
print(X.shape)


(14640, 3000)


In [20]:
# Assuming df is your DataFrame containing the "text" column
first_tweet = df['text'].iloc[0]  # Get the first tweet
cleaned_text = clean_text(first_tweet)  # Apply the clean_text function

# Compare with the options
options = {
    'A': '@ virginamerica @ dhepburn said .',
    'B': '@VirginAmerica What @dhepburn said.',
    'C': 'VirginAmerica What dhepburn said.',
    'D': 'VirginAmerica What dhepburn said',
    'E': 'virginAmerica what dhepburn said.',
    'F': 'virginAmerica what dhepburn said'
}

# Check which option matches the cleaned text
for option, text in options.items():
    if cleaned_text == text:
        print("Option", option, "matches the cleaned text:", text)


Option A matches the cleaned text: @ virginamerica @ dhepburn said .


In [21]:
# Group the DataFrame by the "airline_sentiment" column and describe each group
grouped = df.groupby('airline_sentiment').describe()

# Get the size of the group corresponding to the "negative" sentiment class
negative_count = grouped.loc['negative', ('text', 'count')]

print("Number of instances/rows with 'negative' sentiment class:", negative_count)


Number of instances/rows with 'negative' sentiment class: 9178


In [22]:
# Filter the DataFrame for tweets with the sentiment of "neutral"
neutral_tweets = df[df['airline_sentiment'] == 'neutral']

# Calculate the number of unique tweets
unique_neutral_tweets = neutral_tweets['text'].nunique()

print("Number of unique tweets with 'neutral' sentiment:", unique_neutral_tweets)


Number of unique tweets with 'neutral' sentiment: 3067
